In [1]:
push!(LOAD_PATH, "../src")
include("../src/lanczos.jl")

using .StochasticLanczos
using SparseArrays
using LinearAlgebra
using FastExpm  # For computing matrix exponentials as a benchmark
using Plots

Import the test case. This includes the sparse matrix A, along with the matrix function f(A) = exp(A), and its exact computation

In [2]:
include("test_case_1.jl");

Now test the implementation of the stochastic Lanczos quadrature by comparing the approximate calculation with the exact calculation of $\Omega^\top f(A)\Omega$ where $\Omega\in \mathbb{R}^{n\times b}$ is a matrix whose columns $\omega_i$ are random samples. Note that $b$ is the block size.

In [ ]:
b = 2
Ω = randn(n,b)
Ω = hcat([ ω/norm(ω) for ω in eachcol(Ω) ]...)

In [ ]:
k = 10
block_stochastic_lanczos_quadrature(f, matvecA, Ω, k, "single")

In [ ]:
block_stochastic_lanczos_quadrature(f, matvecA, Ω, k÷b, "block")

In [ ]:
exact = tr(Ω'*exactfA*Ω)